In [48]:
import pandas  as pd 
from itertools import product
from collections import defaultdict
from itertools import groupby
from copy import deepcopy
import os
import csv
import ast
import re

Estado actual: 
* Pandas es muy lento para lo que quiero hacer
* Trabajo actual: separar los desempates en: Primero wins, despues rounds, despues mapas (probablemente no se implemente)
* Potenciales soluciones: Cambiar la manera de armar la tabla a Numpy (deberia ser lo mas rapido por mucho) Ventajas: velocidad y puedo evaluar por filas los resultados 

-------------------------------------------------------------------------

Arranco casi de 0, problemas con la unicacion de memoria de los objetos y se armo un re quilombo. 
Intento 2 diseño enfocado tabla final que tengo y estructurar desde ese punto hacia atras. 

-------------------------------------------------------------------------

24-1-25
Funcionando en trabajo por lotes. 
Dificultades actuales ordenar los batch para procesarlos. Resto de las funcionalidades parecen ok 


In [66]:
teamNameScores = {
                "name": ["KC","TL","BBL","FUT","M8","MKOI"],
                "wins": [2,2,2,2,1,0],
                "loss":[1,1,1,1,2,3],
                "matchWins" : [5,5,5,4,3,0],
                "matchLoss" : [2,3,3,3,5,6],
                "roundWin":[86,87,79,78,80,45],
                "roundLoss":[57,67,77,75,99,80],
                'h2h' : [['M8','MKOI'],
                         ['FUT','BBL'],
                         ['MKOI','KC'],
                         ['M8','MKOI'],
                         ['TL'],
                         [],
                         ]
                  }

roundMatches = {
                "week4" : [["TL","MKOI"], ["M8","BBL"],["KC","FUT"]],
                "week5" : [["BBL","FUT"], ["M8","MKOI"],["TL","KC"]]
                }


# teamNameScores = {
#                 "name": ["FNC","TH","NAVI","VIT","APK","GX"],
#                 "wins": [3,3,2,1,0,0],
#                 "loss":[0,0,1,2,3,3],
#                 "matchWins" : [6,6,4,3,2,2],
#                 "matchLoss" : [1,2,4,4,6,6],
#                 "roundWin":[97,94,87,76,76,55],
#                 "roundLoss":[69,67,76,79,96,98],
#                 'h2h' : [['APK','VIT','NAVI'],
#                          ['VIT','GX','APK'],
#                          ['GX','APK'],
#                          ['GX'],
#                          [],
#                          [],
#                          ]
#                   }

# roundMatches = {
#                 "week4" : [["VIT","APK"], ["GX","FNC"],["NAVI","TH"]],
#                 "week5" : [["GX","APK"], ["VIT","NAVI"],["FNC","TH"]]
#                 }


# teamNameScores = {
#                 "name": ["G2","C9","MIBR","NRG","LEV","FUR"],
#                 "wins": [3,2,2,1,1,0],
#                 "loss":[0,1,1,2,2,3],
#                 "matchWins" : [6,5,5,3,2,1],
#                 "matchLoss" : [1,3,3,5,4,6],
#                 "roundWin":[90,95,96,70,52,45],
#                 "roundLoss":[57,75,90,77,61,88],
#                 'h2h' : [['FUR','NRG','MIBR'],
#                          ['NRG','LEV'],
#                          ['LEV','C9'],
#                          ['FUR'],
#                          ['FUR'],
#                          [],
#                          ]
#                   }

# roundMatches = {
#                 "week4" : [["G2","LEV"], ["FUR","C9"],["NRG","MIBR"]],
#                 "week5" : [["G2","C9"], ["MIBR","FUR"],["LEV","NRG"]]
#                 }

# teamNameScores = {
#                 "name": ["KRU","100T","SEN","2G","EG","LOUD"],
#                 "wins": [3,2,2,1,1,0],
#                 "loss":[0,1,1,2,2,3],
#                 "matchWins" : [6,5,4,3,2,2],
#                 "matchLoss" : [2,3,3,4,4,6],
#                 "roundWin":[87,77,76,71,53,80],
#                 "roundLoss":[65,84,55,81,71,88],
#                 'h2h' : [['LOUD','SEN','2G'],
#                          ['EG','LOUD'],
#                          ['EG','100T'],
#                          ['LOUD'],
#                          ['2G'],
#                          [],
#                          ]
#                   }

# roundMatches = {
#                 "week4" : [["100T","2G"], ["SEN","LOUD"],["KRU","EG"]],
#                 "week5" : [["SEN","2G"], ["KRU","100T"],["EG","LOUD"]]
#                 }

In [50]:
#Esto es raro, siento que estoy dando vueltas en circulo con este diccionario creando los objetos y volviendo a usarlos 
#Lo termino usando con un zip, lo que puede ser peligroso con el orden y las keys
#Agregar h2h ganados aca, voy a tener que modificar como funcionan las fuciones 
teamNameScores = {
                "name": ["BLG","XLG","WOL","TE","JDG","AG"],
                "wins": [3,2,2,2,0,0],
                "loss":[0,1,1,1,3,3],
                "matchWins" : [6,5,4,4,1,1],
                "matchLoss" : [2,2,2,3,6,6],
                "roundWin":[91,83,76,86,69,51],
                "roundLoss":[70,73,70,71,83,89],
                'h2h' : [['AG','JDG','XLG'],
                         ['AG','WOL'],
                         ['JDG','TE'],
                         ['AG','JDG'],
                         [],
                         [],
                         ]
                  }

roundMatches = {
                "week4" : [["BLG","WOL"], ["AG","JDG"],["TE","XLG"]],
                "week5" : [["JDG","XLG"], ["WOL","AG"],["BLG","TE"]]
                }

In [51]:
#Entry vacia para checkear creacion 
teamNameScores = {
    "name": [
        "Kru",
        "Lev",
        "100T",
        "mibr",
        "NRG",
        "EG",
        "FUR",
        "LOUD",
        "2G",
        "GAMERS2",
        "C9",
        "SEN",
    ],
    "wins": [0, 0, 0, 0, 0, 0, 0, 0, 0, 00, 0, 0],
    "loss": [0, 0, 0, 0, 0, 0, 0, 0, 0, 00, 0, 0],
    "matchWins": [0, 0, 0,0, 0, 0,0, 0, 0, 0, 0, 0],
    "matchLoss": [0,0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0],
    "roundWin": [0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0],
    "roundLoss": [0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0],
}

roundMatches = {
    "week11": [
        ["Kru", "Lev"],
        ["100T", "mibr"],
        ["NRG", "EG"],
        ["FUR", "LOUD"],
        ["2G", "GAMERS2"],
        ["C9", "SEN"],
    ],
    "week12": [
        ["Kru", "100T"],
        ["Lev", "mibr"],
        ["NRG", "FUR"],
        ["EG", "LOUD"],
        ["C9", "GAMERS2"],
        ["2G", "SEN"],
    ],
}

In [52]:
def process_for_round(teams):
    previous_results = []
    for i, team in enumerate(teams["name"]):
        temp_dict = {
            "UP": 1,
            "Team": team,
            "wins": teams["wins"][i],
            "matchDelta": teams["matchWins"][i] - teams["matchLoss"][i],
            "h2h": teams['h2h'][i]
        }
        previous_results.append(temp_dict)
    return previous_results

In [53]:
def run_round_delta_with_resume(matches, previous_results, batch_size=512, output_file="results_part", progress_file="progress.txt"):
    match_outcomes = list(product([0, 1, 2, 3], repeat=len(matches)))
    result_mapping = {
        0: (0, 2),
        1: (1, 2),
        2: (2, 1),
        3: (2, 0),
    }

    test_group = defaultdict(list)
    for result in previous_results:
        test_group[result["UP"]].append(result)

    # Leer el progreso guardado
    start_batch = 0
    if os.path.exists(progress_file):
        with open(progress_file, "r") as f:
            start_batch = int(f.read().strip())

    total_batches = len(match_outcomes) // batch_size + 1

    for batch in range(start_batch, total_batches):
        batch_outcomes = match_outcomes[batch * batch_size: (batch + 1) * batch_size]
        new_results = []

        for up, group in test_group.items():
            simul_teams = {
                team_data["Team"]: {
                    "wins": team_data["wins"],
                    "matchDelta": team_data["matchDelta"],
                    'h2h': team_data['h2h']
                }
                for team_data in group
            }

            for combination in batch_outcomes:
                temp_teams = deepcopy(simul_teams)

                for match_index, result in enumerate(combination):
                    team1, team2 = matches[match_index]
                    team1_score, team2_score = result_mapping[result]
                    delta_score = abs(team1_score - team2_score)

                    if result in [2, 3]:
                        temp_teams[team1]["wins"] += 1
                        temp_teams[team1]["matchDelta"] += delta_score
                        temp_teams[team2]["matchDelta"] -= delta_score
                        temp_teams[team1]["h2h"] += [team2]
                    else:
                        temp_teams[team2]["wins"] += 1
                        temp_teams[team2]["matchDelta"] += delta_score
                        temp_teams[team1]["matchDelta"] -= delta_score
                        temp_teams[team2]["h2h"] += [team1]

                new_up = f"{up}-{batch}-{combination}"

                for team, data in temp_teams.items():
                    new_results.append({
                        "UP": new_up,
                        "Team": team,
                        "wins": data["wins"],
                        "matchDelta": data["matchDelta"],
                        'h2h': data['h2h']
                    })

        # Guardado resultados parciales
        output_file_batch = output_file+str(batch)+'.csv'
        df = pd.DataFrame(new_results)
        file_exists = os.path.exists(output_file_batch) and os.path.getsize(output_file_batch) > 0
        df.to_csv(output_file_batch, mode='a', index=False, header=not file_exists)

        # Guardado del progreso
        with open(progress_file, "w") as f:
            f.write(str(batch + 1))

        print(f"Lote {batch + 1}/{total_batches} procesado y guardado.")

    print("Procesamiento completado.")
    os.remove(progress_file) #borrar el progreso si termino de procesar todo 

In [54]:
def groupby_UP(matchresults) -> list:
    group_result = defaultdict(list)
    for result in matchresults:
        group_result[result["UP"]].append(result)
    return group_result


def group_table(teamDict) -> list:
    return [list(group) for _, group in groupby(teamDict, key=lambda x: x[1]["wins"])]


def heriarchy_dict_for_sort(groupby_up_dict: dict) -> dict:
    hierarchy_dict_to_return = {}
    for key in groupby_up_dict.keys():
        hierarchy_dict_to_append = {}
        for team in groupby_up_dict[key]:
            hierarchy_dict_to_append[team["Team"]] = {
                "wins": team["wins"],
                "matchDelta": team["matchDelta"],
                "h2h": team["h2h"],
            }
        hierarchy_dict_to_return[key] = hierarchy_dict_to_append
    return hierarchy_dict_to_return


def sort_dict_by_wins(single_up_round) -> list:
    count = 1  # Esto se puede sacar ?
    sorted_table = sorted(
        single_up_round.items(),
        key=lambda x: (x[1]["wins"], x[1]["matchDelta"]),
        reverse=True,
    )
    grouped_table = group_table(sorted_table)
    count += 1  # Esto se puede sacar ?
    return grouped_table


# Tiebreak Rules
def two_teams_tie(
    tie_teams: list,
) -> (
    list
):  # Esta es rara, falla en tablas incompletas (pueden no haber jugado los dos equipos, pero no deberia falla en la ultima ronda)
    if len(tie_teams) != 2:
        print("not 2 tie teams")
    else:
        team1_h2h_vs_rival = set(tie_teams[0][1]["h2h"]) & set([tie_teams[1][0]])

        if len(team1_h2h_vs_rival) == 0:
            tie_teams = tie_teams[1:] + tie_teams[:1]
    return tie_teams


def three_way_tie(tie_teams: list) -> list:
    if len(tie_teams) != 3:
        print("not 3 way tie")
    else:
        teams = [x[0] for x in tie_teams]
        for index, team in enumerate(tie_teams):  # one team superiority vs the other 2
            team_h2h = set(tie_teams[index][1]["h2h"]) & set(teams)

            if len(team_h2h) == 2:
                pop_team = tie_teams.pop(index)
                remaining_tie = two_teams_tie(tie_teams)
                tie_teams = [pop_team] + remaining_tie

            elif index == 2:
                print("Round diff needed")
    return tie_teams


def tiebreaker(tie_teams):
    amount_tie_teams = len(tie_teams)
    if amount_tie_teams >= 4:
        match_tie_set = set()
        for team in tie_teams:  # This need to be check
            match_tie_set.add(team[1]["matchDelta"])
        if amount_tie_teams != match_tie_set:
            pass

    if amount_tie_teams == 3:
        tie_teams = three_way_tie(tie_teams)

    if amount_tie_teams == 2:  # Esto se puede cambiar y modificar solo la posición
        tie_teams = two_teams_tie(tie_teams)

    return tie_teams or []

In [55]:
def untie_round_save(up_dict_sorted,dict_up_position, key):
    positon = 1
    return_dict = []
    for list_team in up_dict_sorted:
        if len(list_team) == 1:                  
            dict_up_position[list_team[0][0]][str(positon)][0] += 1
            dict_up_position[list_team[0][0]][str(positon)][1].add(key)
            positon += 1
            return_dict += [list_team]
        if len(list_team) > 1:           
            sorted_dict_temp = tiebreaker(list_team)
            for team in sorted_dict_temp:                           
                dict_up_position[team[0]][str(positon)][0] += 1
                dict_up_position[team[0]][str(positon)][1].add(key)
                positon += 1
            return_dict += sorted_dict_temp          
    return return_dict

In [56]:
def transform_csv_2_dict(url):
    with open(url, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        data = []
        for row in reader:
            data.append({
                "UP": row["UP"], 
                "Team": row["Team"],
                "wins": int(row["wins"]),
                "matchDelta": int(row["matchDelta"]),
                "h2h": ast.literal_eval(row["h2h"])  
            })
    return data

In [67]:
test_set_dict = {}
for team in teamNameScores["name"]:
    test_set_dict[team] = {
        "1": [0,set()],
        "2": [0,set()],
        "3": [0,set()],
        "4": [0,set()],
        "5": [0,set()],
        "6": [0,set()],
    }

In [68]:
def process_batch(feed_path, result_path, matches, batch_size=512, output_file="result.csv", progress_file="progress.txt"): #Ojo que ouput result no se esta usando
    # Listar todos los archivos CSV en el directorio
    files_csv = [archivo for archivo in os.listdir(feed_path) if archivo.endswith(".csv")]

    try:
        os.mkdir(result_path)
        print(f"Directory '{result_path}' created successfully.")
    except FileExistsError:
        print(f"Directory '{result_path}' already exists.")
    except PermissionError:
        print(f"Permission denied: Unable to create '{result_path}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

    for file in files_csv:
        file_path = os.path.join(feed_path, file)  
        print(f"Processing file: {file_path}")
        
        try:
            data = transform_csv_2_dict(file_path)
            file_output = os.path.join(result_path, f"processed_{file}")
            
            run_round_delta_with_resume(
                matches=matches,
                previous_results=data,
                batch_size=batch_size,
                output_file=file_output,
                progress_file=progress_file
            )
            
            print(f"File {file} processed and saved to {file_output}.")
        
        except Exception as e:
            print(f"Error processing file {file}: {e}")

In [69]:
teams = teamNameScores
matches = tuple(tuple(match) for match in roundMatches['week4'])
matches2 = tuple(tuple(match) for match in roundMatches['week5'])

In [70]:
#Process de fecha 1 (se pueden mejorar los paths)
previous_results = process_for_round(teams)

run_round_delta_with_resume(matches, previous_results=previous_results, batch_size=512, output_file=".//results//results_batch_test", progress_file=".//results//progress//progress1.txt")

Lote 1/1 procesado y guardado.
Procesamiento completado.


In [71]:
#Aca va match2
feed_path = './/results'
path_results = './/results//fecha_2'

process_batch(feed_path,path_results, matches2, batch_size=512,output_file="result_test.csv",progress_file="progress.txt")

Directory './/results//fecha_2' already exists.
Processing file: .//results\results_batch_test0.csv
Lote 1/1 procesado y guardado.
Procesamiento completado.
File results_batch_test0.csv processed and saved to .//results//fecha_2\processed_results_batch_test0.csv.


In [72]:
#procesar lote de resultados: 
csv_to_process = './/results//fecha_2'

files_csv = [archivo for archivo in os.listdir(csv_to_process) if archivo.endswith(".csv")]

In [73]:
test_set_dict

{'KC': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()]},
 'TL': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()]},
 'BBL': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()]},
 'FUT': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()]},
 'M8': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()]},
 'MKOI': {'1': [0, set()],
  '2': [0, set()],
  '3': [0, set()],
  '4': [0, set()],
  '5': [0, set()],
  '6': [0, set()]}}

In [74]:
for file in files_csv:
    file_path = csv_to_process + '//' + file
    data = transform_csv_2_dict(file_path)
    grouped_data = groupby_UP(data)
    dict_ready_to_sorot = heriarchy_dict_for_sort(grouped_data)
    for key in dict_ready_to_sorot.keys():
        sorted_dict = sort_dict_by_wins(dict_ready_to_sorot[key])
        untie_round_save(sorted_dict,test_set_dict,key)

Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff needed
Round diff

In [75]:
test_set_dict

{'KC': {'1': [1154,
   {'1-0-(2, 2, 2)-0-(2, 2, 1)',
    '1-0-(3, 2, 0)-0-(3, 3, 0)',
    '1-0-(3, 3, 2)-0-(3, 3, 1)',
    '1-0-(0, 1, 2)-0-(0, 0, 0)',
    '1-0-(1, 1, 3)-0-(1, 1, 2)',
    '1-0-(2, 1, 2)-0-(1, 0, 0)',
    '1-0-(3, 3, 1)-0-(2, 0, 0)',
    '1-0-(1, 3, 3)-0-(3, 0, 1)',
    '1-0-(3, 3, 2)-0-(3, 2, 0)',
    '1-0-(2, 3, 3)-0-(2, 3, 0)',
    '1-0-(0, 2, 3)-0-(3, 3, 1)',
    '1-0-(2, 0, 3)-0-(1, 3, 1)',
    '1-0-(1, 3, 2)-0-(1, 2, 0)',
    '1-0-(3, 3, 2)-0-(0, 2, 0)',
    '1-0-(2, 1, 2)-0-(0, 1, 1)',
    '1-0-(2, 3, 0)-0-(2, 1, 0)',
    '1-0-(3, 2, 2)-0-(1, 0, 1)',
    '1-0-(0, 2, 2)-0-(2, 3, 3)',
    '1-0-(0, 0, 3)-0-(1, 1, 1)',
    '1-0-(1, 0, 3)-0-(1, 1, 0)',
    '1-0-(1, 3, 3)-0-(0, 3, 1)',
    '1-0-(2, 3, 3)-0-(3, 3, 1)',
    '1-0-(3, 3, 3)-0-(0, 3, 0)',
    '1-0-(2, 2, 2)-0-(0, 3, 1)',
    '1-0-(1, 2, 3)-0-(3, 0, 1)',
    '1-0-(1, 3, 3)-0-(3, 3, 1)',
    '1-0-(3, 3, 0)-0-(3, 2, 0)',
    '1-0-(0, 1, 2)-0-(0, 1, 3)',
    '1-0-(3, 3, 3)-0-(3, 3, 0)',
    '1-0-(0, 0, 2)-0-(0

In [76]:
#Creo que esto no haace falta en los test
path_file = csv_to_process + '//' + files_csv[0]
data = transform_csv_2_dict(path_file)

In [77]:
groupby_up = groupby_UP(data)

In [78]:
dict_ready_to_sort = heriarchy_dict_for_sort(groupby_up)

In [79]:
dict_ready_to_sort

{'1-0-(0, 0, 0)-0-(0, 0, 0)': {'KC': {'wins': 3,
   'matchDelta': 3,
   'h2h': ['M8', 'MKOI', 'TL']},
  'TL': {'wins': 2, 'matchDelta': -2, 'h2h': ['FUT', 'BBL']},
  'BBL': {'wins': 3, 'matchDelta': 2, 'h2h': ['MKOI', 'KC', 'M8']},
  'FUT': {'wins': 4, 'matchDelta': 5, 'h2h': ['M8', 'MKOI', 'KC', 'BBL']},
  'M8': {'wins': 1, 'matchDelta': -6, 'h2h': ['TL']},
  'MKOI': {'wins': 2, 'matchDelta': -2, 'h2h': ['TL', 'M8']}},
 '1-0-(0, 0, 0)-0-(0, 0, 1)': {'KC': {'wins': 3,
   'matchDelta': 2,
   'h2h': ['M8', 'MKOI', 'TL']},
  'TL': {'wins': 2, 'matchDelta': -1, 'h2h': ['FUT', 'BBL']},
  'BBL': {'wins': 3, 'matchDelta': 2, 'h2h': ['MKOI', 'KC', 'M8']},
  'FUT': {'wins': 4, 'matchDelta': 5, 'h2h': ['M8', 'MKOI', 'KC', 'BBL']},
  'M8': {'wins': 1, 'matchDelta': -6, 'h2h': ['TL']},
  'MKOI': {'wins': 2, 'matchDelta': -2, 'h2h': ['TL', 'M8']}},
 '1-0-(0, 0, 0)-0-(0, 0, 2)': {'KC': {'wins': 2,
   'matchDelta': 0,
   'h2h': ['M8', 'MKOI']},
  'TL': {'wins': 3, 'matchDelta': 1, 'h2h': ['FUT', 'BBL

In [80]:
def sort_dict_by_wins(single_up_round):
    #count = 1
    sorted_table = sorted(single_up_round.items(), key =lambda x: (x[1]["wins"], x[1]["matchDelta"]), reverse=True)
    
    grouped_table = group_table(sorted_table)
    
    #count += 1
    return grouped_table
#Aca tengo que agregar el desempate y los resultados y no crear todo la lista larga? 

In [81]:
def two_teams_tie(tie_teams):
    if len(tie_teams) != 2:
        print("not 2 tie teams")
    else:
        team1_h2h_vs_rival = set(tie_teams[0][1]["h2h"]) & set([tie_teams[1][0]])
        if len(team1_h2h_vs_rival) == 0:
            tie_teams = tie_teams[1:] + tie_teams[:1]

    return tie_teams

In [82]:
def three_way_tie(tie_teams):
    if len(tie_teams) != 3:
        print("not 3 way tie")
    else:
        teams = [x[0] for x in tie_teams]
        for index, team in enumerate(tie_teams):  # one team superiority vs the other 2
            team_h2h = set(tie_teams[index][1]["h2h"]) & set(teams)

            if len(team_h2h) == 2:
                pop_team = tie_teams.pop(index)
                remaining_tie = two_teams_tie(tie_teams)
                tie_teams = [pop_team] + remaining_tie

            elif index == 2:
                pass#print("Round diff needed")

    return tie_teams

In [83]:
def tiebreaker(tie_teams):
    amount_tie_teams = len(tie_teams)
    if amount_tie_teams >= 4:
        match_tie_set = set()
        for team in tie_teams: #This need to be check
            match_tie_set.add(team[1]["matchDelta"])  
        if amount_tie_teams != match_tie_set:
            pass 
        
    if amount_tie_teams == 3:
        tie_teams = three_way_tie(tie_teams)
        
    if amount_tie_teams == 2:  # Esto se puede cambiar y modificar solo la posición
        
        tie_teams = two_teams_tie(tie_teams)
        
    return tie_teams or []

In [84]:
def untie(up_dict_sorted,dict_up_position):
    positon = 1
    return_dict = []
    for list_team in up_dict_sorted:
        if len(list_team) == 1:            
            dict_up_position[list_team[0][0]][str(positon)] += 1
            positon += 1
            return_dict += [list_team]
        if len(list_team) > 1:           
            sorted_dict_temp = tiebreaker(list_team)
            for team in sorted_dict_temp:             
                dict_up_position[team[0]][str(positon)] += 1
                positon += 1
            return_dict += sorted_dict_temp          
    return return_dict

In [85]:

def extract_parentheses_values(input_string):
    # Valores dentro de parentesis
    pattern = r'\((.*?)\)'
    matches = re.findall(pattern, input_string)
    # Convertir los grupos grupo capturado en una lista de enteros
    results = [list(map(int, match.split(','))) for match in matches]
    return results

In [86]:
def create_nested_dict(start, end,matches_per_week=3):
    match_results = {}
    for i in range(start, end + 1):
        match_results[str(i)] = {str(j): [] for j in range(matches_per_week)} # ACA OJO CON EL 4 HARDCODEADO
    return match_results

# Crear el diccionario desde 1 hasta 10 como ejemplo
matchs_per_week = 3
nested_dict = create_nested_dict(1, 6,matchs_per_week)

In [87]:
#Probablemente aca me conviene pasar a sql, armar la base de datos y trabajarla desde sql directo
#Checkear que es mas rapido, correr 2 veces el for con un extracted element particular o el enumarte de extracted_element + mas el otro for
#El enumarated(extracted_element) con el for evaluando los 0 1 es mas escalable 

def create_position_dicts(team, position_set_dict,nested_dict_range=[1,4],final_poss=4):
    result_mapping = {
        0: '0-2',
        1: '1-2',
        2: '2-1',
        3: '2-0',
    }
    nested_dict = create_nested_dict(nested_dict_range[0], nested_dict_range[1],matchs_per_week)
    nested_dict_match2 = create_nested_dict(nested_dict_range[0], nested_dict_range[1],matchs_per_week)
    for position in position_set_dict[team]:
        if int(position) < final_poss:
            for element in position_set_dict[team][position][1]:
                extracted_element = extract_parentheses_values(element)
                for number, result in enumerate(extracted_element[0]): 
                    nested_dict[str(position)][str(number)].append(
                        (matches[number][0], result_mapping[result], matches[number][1])
                    )
                for number, result in enumerate(extracted_element[1]):
                    nested_dict_match2[str(position)][str(number)].append(
                        (matches2[number][0], result_mapping[result], matches2[number][1])
                    )
    return nested_dict, nested_dict_match2

In [88]:
def df_with_positions(weeek1,week2,team):
    postion = ['position','team']
    week1columns = ['week1_match_1','week1_match_2','week1_match_3']
    week2columns = ['week2_match_1','week2_match_2','week2_match_3']

    columns_df = postion + week1columns + week2columns

    df_base = pd.DataFrame(columns=columns_df)

    for key in weeek1.keys():
      
        df_week1 = pd.DataFrame.from_dict(weeek1[key])
        
        df_week1.columns = week1columns
        df_week2 = pd.DataFrame.from_dict(week2[key])
        df_week2.columns = week2columns
        
        df_combined = pd.concat([df_week1, df_week2], axis=1)
        df_combined['position'] = key
        df_combined['team'] = team
       

        df_base = pd.concat([df_base,df_combined],ignore_index=True)
    return df_base

In [89]:
teams = teamNameScores['name']
len_teams = len(teams) + 1

count = 0
for team in teams:
    week1_dict_to_df, week2_dict_to_df = create_position_dicts(team,test_set_dict,[1,len_teams],len_teams)
    df_g2 = df_with_positions(week1_dict_to_df,week2_dict_to_df,team)
    df_g2.to_csv(path_or_buf=team+'.csv',index=False)
    matches_columns = [col for col in df_g2.columns if 'match' in col] 
    df_melted = df_g2.melt(id_vars=['position'], value_vars=matches_columns, value_name='match').drop('variable', axis=1)
    temp_result = df_melted.groupby(['position', 'match']).size().reset_index(name='count')
    temp_result['team'] = team
    
    if count == 0:
         result_df = temp_result.copy()
         count += 1
    else:
        result_df = pd.concat([result_df,temp_result],ignore_index=True)

In [90]:
result_df.to_csv(path_or_buf='compilado.csv',index=False)

In [32]:
week1_dict_to_df, week2_dict_to_df = create_position_dicts('KRU',[1,len_teams],len_teams)

TypeError: 'int' object is not subscriptable

In [ ]:
df_g2 = df_with_positions(week1_dict_to_df,week2_dict_to_df,'KRU')

In [93]:
df_g2

,position,team,week1_match_1,week1_match_2,week1_match_3,week2_match_1,week2_match_2,week2_match_3
